In [1]:
%pylab inline
import Stracers as st
import sys
sys.path.append('../MW_anisotropy/code/')
sys.path.append('../MW_anisotropy/code/densities/')

import density_tools
import reading_snapshots
from pygadgetreader import *



Populating the interactive namespace from numpy and matplotlib


In [5]:
def all_host_particles(xyz, vxyz, pids, pot, mass, N_host_particles):
    """
    Function that return the host and the sat particles
    positions and velocities.

    Parameters:
    -----------
    xyz: snapshot coordinates with shape (n,3)
    vxys: snapshot velocities with shape (n,3)
    pids: particles ids
    Nhost_particles: Number of host particles in the snapshot
    Returns:
    --------
    xyz_mw, vxyz_mw, xyzlmc, vxyz_lmc: coordinates and velocities of
    the host and the sat.

    """
    sort_indexes = np.sort(pids)
    N_cut = sort_indexes[N_host_particles]
    host_ids = np.where(pids<N_cut)[0]
    return xyz[host_ids], vxyz[host_ids], pids[host_ids], pot[host_ids], mass[host_ids]

In [6]:
def finding_indices(x, y):
    """
    Return the indicies in which the ids of the two snapshots are the same.
    """
    
    indices = np.zeros(len(x))
    for i in range(len(x)):
        indices[i] = (np.where(y==x[i])[0])
        
    return indices

In [11]:
def energies(snap):
    pp= readsnap(snap, 'pos', 'dm')
    vv= readsnap(snap, 'vel', 'dm')
    massarr= readsnap(snap, 'mass', 'dm')
    Epp = readsnap(snap, 'pot', 'dm')
    ids = readsnap(snap, 'pid', 'dm')
    
    MW_pos, MW_vel, MW_pot, MW_ids, MW_mass = all_host_particles(pp, vv, ids, Epp, massarr, 100000000)

    rr=np.sqrt(MW_pos[:,0]**2+MW_pos[:,1]**2+MW_pos[:,2]**2)
    rcut = 300
    r_cut = index = np.where((rr<rcut))[0]

    MW_pos = MW_pos[r_cut]
    rr = rr[r_cut]
    MW_vel = MW_vel[r_cut]

    massarr = massarr[r_cut]
    Epp = Epp[r_cut]
    ids = ids[r_cut]

    #partmass=massarr[0]*1e10 #generated the halo particles as "bulge"-type in Gadget file
    v2=MW_vel[:,0]**2+MW_vel[:,1]**2+MW_vel[:,2]**2
    Ekk=0.5*v2
    
    return rr, Ekk, Epp, ids, massarr


In [ ]:
rr_init, Ekk_init, Epp_init, ids_init, massarr_init = energies('./halos/MWLMC6_100M_new_b0_2_000')

Returning DM    Positions
Returning DM    Velocities
Returning DM    Mass
Returning DM    Potential
Returning DM    Particle IDs


In [ ]:
rr_fut, Ekk_fut, Epp_fut, ids_fut, massarr_fut = energies('./halos/MWLMC6_100M_new_b0_2_111')

In [ ]:
indices_ids_fut = finding_indices(ids_init, ids_fut)

In [ ]:
wh, w_ids_h = st.weight_triaxial(rr, Ekk, Epp, ids, partmass, 0.01, 100, 1, 'Hernquist', [25])

In [ ]:
st_mass, st_w_ids = st.stellar_quantity(wh, ids, w_ids_h, massarr*1E10)

In [ ]:
print(sum(st_mass), sum(massarr*1E10))

In [ ]:
rho_mw_st = density_tools.density_nn(pp, 200, 100, 10, 1000, mass=st_mass)

In [ ]:
def rho_enclosed(r_posh, rmin, rmax, nbins, mass):
    r = np.linspace(rmin, rmax, nbins-1)
    rho = np.zeros(nbins-1)

    # Loop over the radial bins.
    for i in range(1, len(r)):
        indexh = np.where((r_posh<r[i]) & (r_posh>r[i-1]))[0]

        rho[i-1] = (3*(np.sum(mass[indexh]))) / (4*np.pi*(r[i]**3-r[i-1]**3))

    return r, rho

In [ ]:
rho_mw_dm = density_tools.density_nn(pp, 200, 100, 10, 1000, mass=massarr*1E10)

In [ ]:
r_r, rho_mw_st_r = rho_enclosed(rr, 1, 300, 1000, st_mass)
r_r, rho_mw_dm_r = rho_enclosed(rr, 1, 300, 1000, massarr*1E10)

In [ ]:
loglog(r_r, rho_mw_dm_r)
loglog(r_r, rho_mw_st_r)

In [ ]:
density_tools.density_peaks(np.log(rho_mw_st)/np.max(np.log(rho_mw_st)), xmin=-100, xmax=100, 
                            ymin=-100, ymax=100, fsize=(6, 4.5),  levels=np.arange(0, 1.05, 0.1))

In [ ]:
density_tools.density_peaks(np.log(rho_mw_dm)/np.max(np.log(rho_mw_dm)), xmin=-100, xmax=100, 
                            ymin=-100, ymax=100, fsize=(6, 4.5),  levels=np.arange(0, 1.05, 0.1))

In [ ]:
x = np.array([1, 3, 5, 7, 9])
y = x[::-1]
print(x, y)

In [ ]:
finding_indices(x, y)